# Back Propagation

In order to understand backpropagation, you will implement a backpropagation algorithm for a neural network from scratch. <br/>
Backpropagation can be used for both classification and regression problems, we will focus on classification in
this exercise.

#### Exercise 1 : initialization of the network

In [ ]:
from random import random, seed, shuffle

In [ ]:
def init_network(n_inputs, n_hidden, n_outputs): 
    network = list()

    # hidden layer 
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)

    # output layer 
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)

    return network


#### Exercise 2 : 

In [ ]:
seed(1)
net = init_network(2,4,6)
for layer in net: 
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}, {'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}]
[{'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014, 0.7215400323407826, 0.22876222127045265]}, {'weights': [0.9452706955539223, 0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966]}, {'weights': [0.9391491627785106, 0.38120423768821243, 0.21659939713061338, 0.4221165755827173, 0.029040787574867943]}, {'weights': [0.22169166627303505, 0.43788759365057206, 0.49581224138185065, 0.23308445025757263, 0.2308665415409843]}, {'weights': [0.2187810373376886, 0.4596034657377336, 0.28978161459048557, 0.021489705265908876, 0.8375779756625729]}, {'weights': [0.5564543226524334, 0.6422943629324456, 0.1859062658947177, 0.9925434121760651, 0.8599465287

### Forward Propagation

#### Exercise 3 : Neuron activation

In [ ]:
# Weight is a network weight
# Input is an input (can be line of a dataset, should be a vector of the same size of the number of nodes in the network)
# Bias is a special weight that has no input to multiply with

# activation = sum(weight_i * input_i) + bias

In [ ]:
# weights is a list of given weights
# inputs is a list of inputs

def activate(weights, inputs): 
    # To simplify, the bias = the last weight of weights
    activation = weights[-1]
    for i in range(len(weights) - 1): 
        activation += weights[i] * inputs[i]
    return activation

#### Exercise 4 : Neuron transfer

In [ ]:
from math import exp

def transfer(activation): 
    return 1.0/(1.0 + exp(-activation))

#### Exercise 5 : Forward propagation

In [ ]:
# Forward propagate input to a network output
def forward_propagate(network, row): 
    inputs = row
    for layer in network: 
        new_inputs = []
        for neuron in layer: 
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

#### Exercise 6 : Testing our forward propagation

In [ ]:
seed(1)
network = init_network(3, 2, 2)
row = [1, 0, 0]
output = forward_propagate(network, row)

output

[0.7138015773148616, 0.7020509238862008]

### Back Propagation Error

#### Exercise 7 : Transfer derivative

In [ ]:
# if transfer function is sigmoid 
def transfer_derivative(output): 
    return output * (1.0 - output)

#### Exercise 8 : Error Backpropagation

In [ ]:
# error = (expected - output) * transfer_derivative(output)

In [ ]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected): 
    for i in reversed(range(len(network))): 
        layer = network[i]
        errors = list()
        if i != len(network) - 1 : 
            for j in range(len(layer)): 
                error = 0.0
                for neuron in network[i+1]: 
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else: 
            for j in range(len(layer)): 
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)): 
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

#### Exercise 9 : Test backward propagate error

In [ ]:
expected = [0.7138015773148616, 0.7020509238862008]
backward_propagate_error(network, expected)

In [ ]:
network

[[{'weights': [0.13436424411240122,
    0.8474337369372327,
    0.763774618976614,
    0.2550690257394217],
   'output': 0.5961462631100435,
   'delta': 0.0},
  {'weights': [0.49543508709194095,
    0.4494910647887381,
    0.651592972722763,
    0.7887233511355132],
   'output': 0.7831568031528053,
   'delta': 0.0}],
 [{'weights': [0.0938595867742349, 0.02834747652200631, 0.8357651039198697],
   'output': 0.7138015773148616,
   'delta': 0.0},
  {'weights': [0.43276706790505337, 0.762280082457942, 0.0021060533511106927],
   'output': 0.7020509238862008,
   'delta': 0.0}]]

#### Exercise 10 : Update weights

In [ ]:
# weight = weight + learning rate * error * input

where **weight** is a given weight, <br/>
**learning rate** is a parameter that you must
specify, <br/>
**error** is the error calculated by the backpropagation procedure for the
neuron and <br/>
**input** is the input value that caused the error.

In [ ]:
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

#### Exercise 11 : Train network

In [ ]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

#### Exercise 12 : Define a dataset and test

In [ ]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

In [ ]:
seed(1)
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = init_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

> epoch=0, lrate=0.500, error=6.350
> epoch=1, lrate=0.500, error=5.531
> epoch=2, lrate=0.500, error=5.221
> epoch=3, lrate=0.500, error=4.951
> epoch=4, lrate=0.500, error=4.519
> epoch=5, lrate=0.500, error=4.173
> epoch=6, lrate=0.500, error=3.835
> epoch=7, lrate=0.500, error=3.506
> epoch=8, lrate=0.500, error=3.192
> epoch=9, lrate=0.500, error=2.898
> epoch=10, lrate=0.500, error=2.626
> epoch=11, lrate=0.500, error=2.377
> epoch=12, lrate=0.500, error=2.153
> epoch=13, lrate=0.500, error=1.953
> epoch=14, lrate=0.500, error=1.774
> epoch=15, lrate=0.500, error=1.614
> epoch=16, lrate=0.500, error=1.472
> epoch=17, lrate=0.500, error=1.346
> epoch=18, lrate=0.500, error=1.233
> epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.002627965285

#### Exercise 13 : Predict

In [ ]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

#### Exercise 14 : Test the prediction

In [ ]:
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


#### Exercise 15 and 16 : Import dataset

In [ ]:
import csv
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = csv.reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
			
	for row in range(len(dataset)): 
		dataset[row] = dataset[row][0].split('\t')
		for i in range(len(dataset[row])): 
			dataset[row][i] = float(dataset[row][i].strip())
	return dataset

In [ ]:
seed(1)
dataset = load_csv('seeds_dataset.csv')
dataset

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 1.0],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 1.0],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 1.0],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 1.0],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 1.0],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, 1.0],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, 1.0],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 1.0],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, 1.0],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, 1.0],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, 1.0],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, 1.0],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, 1.0],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, 1.0],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, 1.0],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, 1.0],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, 1.0],
 [15.69, 14.75, 0.9058, 5.

#### Exercise 17 : Shuffle dataset and normalize columns

In [ ]:
shuffle(dataset)
dataset

[[14.92, 14.43, 0.9006, 5.384, 3.412, 1.142, 5.088, 1.0],
 [18.98, 16.66, 0.859, 6.549, 3.67, 3.691, 6.498, 2.0],
 [18.94, 16.32, 0.8942, 6.144, 3.825, 2.908, 5.949, 2.0],
 [12.02, 13.33, 0.8503, 5.35, 2.81, 4.271, 5.308, 3.0],
 [18.88, 16.26, 0.8969, 6.084, 3.764, 1.649, 6.109, 2.0],
 [12.05, 13.41, 0.8416, 5.267, 2.847, 4.988, 5.046, 3.0],
 [12.46, 13.41, 0.8706, 5.236, 3.017, 4.987, 5.147, 3.0],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, 1.0],
 [19.14, 16.61, 0.8722, 6.259, 3.737, 6.682, 6.053, 2.0],
 [14.8, 14.52, 0.8823, 5.656, 3.288, 3.112, 5.309, 1.0],
 [16.12, 15.0, 0.9, 5.709, 3.485, 2.27, 5.443, 1.0],
 [17.32, 15.91, 0.8599, 6.064, 3.403, 3.824, 5.922, 2.0],
 [18.81, 16.29, 0.8906, 6.272, 3.693, 3.237, 6.053, 2.0],
 [11.02, 13.0, 0.8189, 5.325, 2.701, 6.735, 5.163, 3.0],
 [12.72, 13.57, 0.8686, 5.226, 3.049, 4.102, 4.914, 1.0],
 [16.2, 15.27, 0.8734, 5.826, 3.464, 2.823, 5.527, 1.0],
 [10.93, 12.8, 0.839, 5.046, 2.717, 5.398, 5.045, 3.0],
 [18.55, 16.22, 0.8865, 6.15

In [ ]:
def minmax(dataset): 
    max_per_col, min_per_col, diff = [], [], []
    for i in range(len(dataset[0])):
        column = [] 
        for j in range(len(dataset)): 
            column.append(dataset[j][i])
        max_per_col.append(max(column))
        min_per_col.append(min(column))
        diff.append(max(column) - min(column))
    return (min_per_col, max_per_col, diff)

In [ ]:
def minmax_normalization(dataset): 
    normalized_dataset = []
    mini, maxi, diff = minmax(dataset)
    for i in range(len(dataset)): 
        value = []
        for j in range(len(dataset[0]) - 1): 
            # normalized_value = value - min / max - min
            value.append((dataset[i][j] - mini[j]) / diff[j])
        value.append(int(dataset[i][len(dataset[0])-1]) -1)
        normalized_dataset.append(value)
    return normalized_dataset

In [ ]:
normalized_dataset = minmax_normalization(dataset)
normalized_dataset

[[0.408876298394712,
  0.4173553719008264,
  0.8393829401088925,
  0.2730855855855858,
  0.5573770491803277,
  0.04900596809216086,
  0.2801575578532743,
  0],
 [0.7922568460812087,
  0.878099173553719,
  0.4618874773139742,
  0.9290540540540544,
  0.7412687099073412,
  0.380436619901442,
  0.9743968488429348,
  1],
 [0.7884796978281399,
  0.8078512396694215,
  0.7813067150635207,
  0.7010135135135136,
  0.8517462580185317,
  0.27862798892197277,
  0.7040866568193008,
  1],
 [0.13503305004721433,
  0.19008264462809915,
  0.38294010889292124,
  0.25394144144144126,
  0.128296507483963,
  0.4558504206269748,
  0.38847858197932045,
  2],
 [0.7828139754485363,
  0.7954545454545457,
  0.8058076225045374,
  0.6672297297297296,
  0.8082679971489661,
  0.11492803182982488,
  0.7828655834564254,
  1],
 [0.13786591123701614,
  0.2066115702479339,
  0.3039927404718692,
  0.20720720720720742,
  0.15466856735566645,
  0.5490774811790559,
  0.25947808961102914,
  2],
 [0.1765816808309727,
  0.206611

In [ ]:
def split_train_test_set(dataset, train_set_percentage = 0.8): 
    split = int(len(dataset) * train_set_percentage)
    return (dataset[:split], dataset[split:])

In [ ]:
train_set, test_set = split_train_test_set(normalized_dataset)
len(train_set), len(test_set)

(168, 42)

In [ ]:
train_set[:3]

[[0.408876298394712,
  0.4173553719008264,
  0.8393829401088925,
  0.2730855855855858,
  0.5573770491803277,
  0.04900596809216086,
  0.2801575578532743,
  0],
 [0.7922568460812087,
  0.878099173553719,
  0.4618874773139742,
  0.9290540540540544,
  0.7412687099073412,
  0.380436619901442,
  0.9743968488429348,
  1],
 [0.7884796978281399,
  0.8078512396694215,
  0.7813067150635207,
  0.7010135135135136,
  0.8517462580185317,
  0.27862798892197277,
  0.7040866568193008,
  1]]

#### Exercise 18 : Initialize network

In [ ]:
n_inputs = len(train_set[0]) - 1
n_outputs = len(set([row[-1] for row in train_set]))
seed_network = init_network(n_inputs, 4, n_outputs)

n_inputs, n_outputs

(7, 3)

In [ ]:
train_network(seed_network, train_set, 2, 500, n_outputs)

> epoch=0, lrate=2.000, error=113.448
> epoch=1, lrate=2.000, error=67.660
> epoch=2, lrate=2.000, error=50.436
> epoch=3, lrate=2.000, error=37.101
> epoch=4, lrate=2.000, error=30.736
> epoch=5, lrate=2.000, error=27.057
> epoch=6, lrate=2.000, error=24.370
> epoch=7, lrate=2.000, error=22.664
> epoch=8, lrate=2.000, error=21.488
> epoch=9, lrate=2.000, error=20.483
> epoch=10, lrate=2.000, error=19.612
> epoch=11, lrate=2.000, error=18.868
> epoch=12, lrate=2.000, error=18.144
> epoch=13, lrate=2.000, error=17.462
> epoch=14, lrate=2.000, error=16.863
> epoch=15, lrate=2.000, error=16.295
> epoch=16, lrate=2.000, error=16.008
> epoch=17, lrate=2.000, error=15.670
> epoch=18, lrate=2.000, error=14.865
> epoch=19, lrate=2.000, error=13.961
> epoch=20, lrate=2.000, error=13.619
> epoch=21, lrate=2.000, error=13.281
> epoch=22, lrate=2.000, error=12.990
> epoch=23, lrate=2.000, error=12.694
> epoch=24, lrate=2.000, error=12.467
> epoch=25, lrate=2.000, error=12.401
> epoch=26, lrate=2.0

In [ ]:
for layer in seed_network:
	print(layer)

[{'weights': [-5.770820437781925, -8.646713130372873, -6.401225276687946, -13.236675050442985, -4.456205218457874, 12.518354780836638, 36.77577262988843, -4.258921270450485], 'output': 0.7187570877257409, 'delta': -3.777591925557755e-07}, {'weights': [12.423755098009897, 10.025270064488765, -3.1187318745936374, -12.35067365242796, 7.895759032933681, 12.092565282891966, 17.716460632735505, -23.065514967568333], 'output': 2.2290815526217296e-07, 'delta': -9.385381483218457e-13}, {'weights': [10.741888622939245, 12.523243245810391, -7.416253188808853, 14.005133953664991, 2.1732883748337515, 2.245604626123859, -13.387246187019187, -3.218776713554402], 'output': 0.0030778568021839225, 'delta': -1.9998352991144653e-09}, {'weights': [6.905873879751171, 7.357686963722843, -3.141294209240534, 5.5986028694432175, 2.495356024131931, -0.2805490194809506, -0.2299326140758385, -7.304968345572325], 'output': 0.0004784990079998191, 'delta': -1.933835871841999e-09}]
[{'weights': [-12.7219943495794, -15

#### Exercise 19 : Predict the test set on the trained network

In [ ]:
count, mse = 0, 0
for row in test_set:
	prediction = predict(seed_network, row)
	if prediction == row[-1]: 
		count += 1
	print(f'Expected={row[-1]}, Got={prediction}')

Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=0
Expected=2, Got=2
Expected=1, Got=1
Expected=0, Got=0
Expected=2, Got=2
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=0, Got=0
Expected=0, Got=0
Expected=2, Got=2
Expected=1, Got=1
Expected=0, Got=0
Expected=2, Got=2
Expected=0, Got=0
Expected=2, Got=2
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=0, Got=0
Expected=2, Got=2
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=1, Got=1
Expected=1, Got=1
Expected=2, Got=2
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=2, Got=2
Expected=2, Got=0
Expected=2, Got=2
Expected=0, Got=0


In [ ]:
print(f'Accuracy : {count / len(test_set)}')

Accuracy : 0.9285714285714286


In [1]:
import numpy as np
MSE = np.square(np.subtract(test_set[-1],prediction)).mean() 
print(f'Mean Squarred Error: {MSE}')

NameError: name 'test_set' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=62eda4a8-d26a-4a56-8c6c-5473bb99037f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>